# WISE2MBH Tutorial

In [23]:
import wise2mbh as wm
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import random
no_data = 9876543

In [8]:
allwise = Table.read('../samples/AllWISE_sample.fits')
allwise

ONAME,RA,DEC,Z,T,NED_TYPE,AllWISE,RAJ2000,DEJ2000,eeMaj,eeMin,eePA,W1mag,W2mag,W3mag,W4mag,Jmag,Hmag,Kmag,e_W1mag,e_W2mag,e_W3mag,e_W4mag,e_Jmag,e_Hmag,e_Kmag,ID,ccf,ex,var,qph,pmRA,e_pmRA,pmDE,e_pmDE,d2M,angDist
bytes107,float64,float64,float64,float64,bytes7,bytes19,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,bytes4,bytes1,bytes4,bytes4,int32,int32,int32,int32,float32,float64
WISEAJ112428.14-363548.5,171.1172539,-36.5968225,0.2,9876543.0,Unknown,J112428.14-363548.5,171.1172539,-36.5968225,0.0427,0.0415,130.3,13.449,12.486,9.394,6.977,15.185,14.474,14.293,0.025,0.024,0.033,0.084,0.056,0.061,0.075,1716036401351015875,0000,0,1122,AAAA,160,54,42,55,2.073,1e-06
WISEAJ144229.68-435343.2,220.6236974,-43.8953408,0.3,9876543.0,Unknown,J144229.68-435343.2,220.6236974,-43.8953408,0.1285,0.1177,11.9,15.975,14.663,10.911,7.801,--,--,--,0.055,0.06,0.118,0.195,--,--,--,2205044001351034712,0000,0,011n,AABB,412,466,-34,505,--,1e-06
WISEAJ163728.75+290335.9,249.3698107,29.0599733,0.2,9876543.0,Unknown,J163728.75+290335.9,249.3698107,29.0599733,0.0692,0.0643,4.9,15.637,13.775,9.703,7.347,--,--,--,0.04,0.034,0.038,0.124,--,--,--,2485128701351042160,0000,0,0011,AAAB,-116,295,-250,278,--,2e-06
WISEAJ184321.55-621724.8,280.8398056,-62.2902233,0.2,9876543.0,Unknown,J184321.55-621724.8,280.8398056,-62.2902233,0.0722,0.0689,38.8,14.624,13.762,11.04,8.519,--,--,--,0.036,0.035,0.106,--,--,--,--,2810062101351018362,0000,0,nnnn,AAAU,--,--,--,--,--,5e-06
WISEAJ183203.33+633041.5,278.0139066,63.5115391,0.3,9876543.0,Unknown,J183203.33+633041.5,278.0139066,63.5115391,0.0662,0.0627,129.3,15.811,14.794,11.508,8.357,--,--,--,0.032,0.035,0.086,0.118,--,--,--,2774163601351028837,0000,0,11nn,AAAB,155,233,-229,248,--,7e-06
WISEAJ000517.26-341452.8,1.321926,-34.2480098,0.3,9876543.0,Unknown,J000517.26-341452.8,1.321926,-34.2480098,0.0752,0.0713,141.6,15.1,14.143,10.839,7.887,--,--,--,0.035,0.04,0.114,0.189,--,--,--,18034901351061482,0000,0,110n,AABB,289,256,-282,261,--,7e-06
WISEAJ041514.13-810745.9,63.8088863,-81.1294387,0.2,9876543.0,Unknown,J041514.13-810745.9,63.8088863,-81.1294387,0.0457,0.0449,162.3,14.499,13.543,10.429,7.558,--,--,--,0.025,0.026,0.048,0.099,--,--,--,681081801351062155,0000,0,110n,AAAA,260,66,92,69,--,8e-06
WISEAJ150206.92+781622.6,225.5288639,78.2729584,0.3,9876543.0,Unknown,J150206.92+781622.6,225.5288639,78.2729584,0.0886,0.0831,95.1,15.959,15.085,12.15,9.497,--,--,--,0.037,0.05,0.202,0.512,--,--,--,2232178701351000385,0000,0,11nn,AABC,-761,305,100,285,--,9e-06
WISEAJ050357.41-282633.0,75.989223,-28.4425115,0.2,9876543.0,Unknown,J050357.41-282633.0,75.989223,-28.4425115,0.1182,0.0847,153.4,15.102,14.292,11.284,8.529,--,--,--,0.045,0.045,0.125,0.332,--,--,--,754028801351050034,0000,0,nnnn,AABB,--,--,--,--,--,1.1e-05


Clening process of the sample from AllWISE

In [3]:
size_before_depure = len(allwise)
print(f'Size before depure: {size_before_depure}')

allowed_z = allwise[allwise['Z']!=0]                                                            #Masking of null redshifts
qph_list = allwise['qph'].tolist()                                                              #Extract quality flags
null_phot = ['X', 'Z']                                                                          #Null and uplim values in W bands
not_null_quality = [not any(letter in qph[:-1] for letter in null_phot) for qph in qph_list]    #Test quality flags

allwise = allowed_z[not_null_quality].copy()                                                    #Keep non-null values in quality

size_after_depure = len(allwise)
print(f'Size after depure: {size_after_depure}')

Size before depure: 77712
Size after depure: 77687


Detections with 'U' flag in qph do not contain errors on the WISE bands, so better to set them to a single value, in this case we'll set them to 0

In [11]:
allwise['e_W1mag'] = allwise['e_W1mag'].filled(0)                 #Replace null errors in WISE bands
allwise['e_W2mag'] = allwise['e_W2mag'].filled(0)
allwise['e_W3mag'] = allwise['e_W3mag'].filled(0)

Since this algorithm have a Monte Carlo implementation for error propagation, we recommend to take slices for big sample (rows>200), so for this tutorial, we'll use a random slice from the AllWISE sample.

In [15]:
random_indices = random.sample(range(len(allwise)), 1000)
new_allwise = allwise[random_indices]
new_allwise

ONAME,RA,DEC,Z,T,NED_TYPE,AllWISE,RAJ2000,DEJ2000,eeMaj,eeMin,eePA,W1mag,W2mag,W3mag,W4mag,Jmag,Hmag,Kmag,e_W1mag,e_W2mag,e_W3mag,e_W4mag,e_Jmag,e_Hmag,e_Kmag,ID,ccf,ex,var,qph,pmRA,e_pmRA,pmDE,e_pmDE,d2M,angDist
bytes107,float64,float64,float64,float64,bytes7,bytes19,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,bytes4,bytes1,bytes4,bytes4,int32,int32,int32,int32,float32,float64
LAMOST419804077,166.63744,51.72315,0.1924653,9876543.0,Galaxy,J110632.98+514323.3,166.6374469,51.723157,0.0601,0.0582,6.0,14.32,14.11,12.663,8.871,15.97,15.171,14.752,0.027,0.039,0.498,--,0.079,0.093,0.098,1668151401351045072,0000,0,11nn,AACU,123,123,337,134,0.447,0.029469
SDSSJ230835.71-021237.6,347.1488055,-2.2104537,0.29505232,9876543.0,Unknown,J230835.71-021237.6,347.1488177,-2.2104584,0.0621,0.06,116.1,14.152,13.88,12.462,8.859,16.44,15.899,14.883,0.028,0.038,0.501,--,0.128,0.21,0.134,3479001601351010475,0000,0,00nn,AACU,-331,137,18,137,0.051,0.046894
GAMA202541,130.14027,-0.3927695,0.27082267,9876543.0,Unknown,J084033.66-002334.0,130.140263,-0.3927814,0.092,0.0868,6.4,15.054,14.765,12.558,8.981,--,--,--,0.036,0.066,0.479,0.489,--,--,--,1300100001351018843,d000,0,00nn,AACC,238,239,-139,261,--,0.049701
SDSS2427584860848678912,229.42386,21.6632,0.17358738,9876543.0,Galaxy,J151741.72+213947.5,229.4238594,21.6631973,0.0564,0.0554,154.7,14.337,14.105,12.267,9.453,16.004,15.472,15.145,0.027,0.035,0.0,--,0.081,0.097,0.163,2292121201351048102,0000,0,11nn,AAUU,15,65,23,70,0.557,0.010029
SDSSJ151450.41+042022.3,228.710061,4.3395499,0.46122,9876543.0,Unknown,J151450.41+042022.3,228.710053,4.3395448,0.0794,0.0765,178.2,14.913,14.929,13.028,9.561,--,--,--,0.032,0.069,0.0,--,--,--,--,2293104501351011865,0000,0,11nn,AAUU,180,199,-127,217,--,0.034237
WISEAJ083223.15+053741.0,128.0965051146,5.6280716887,0.134471,9876543.0,Unknown,J083223.15+053741.0,128.0964943,5.6280801,0.0933,0.0889,168.1,15.175,14.877,10.731,8.558,--,--,--,0.036,0.073,0.134,--,--,--,--,1275106001351020566,0000,0,000n,AABU,37,318,133,330,--,0.049192
SDSS1928779854325508096,222.13271,11.07853,0.1667336,9876543.0,Galaxy,J144831.84+110442.6,222.1327057,11.0785255,0.059,0.0575,178.7,14.392,14.116,12.162,8.794,16.543,15.582,14.939,0.027,0.038,0.305,--,0.176,0.169,0.149,2215110601351053063,0000,0,11nn,AABU,34,70,75,77,0.17,0.02228
SDSSJ110346.06+470240.7,165.94192049999998,47.0446576,0.4791,9876543.0,Unknown,J110346.05+470240.8,165.941916,47.0446675,0.1031,0.0954,15.0,15.402,15.249,12.698,9.071,--,--,--,0.038,0.082,0.0,--,--,--,--,1658146901351031769,0000,0,11nn,AAUU,192,293,235,318,--,0.037258
SDSS2638117959691692032,151.70067,24.27225,0.19277637,9876543.0,Galaxy,J100648.15+241620.0,151.7006567,24.2722441,0.0627,0.0592,173.7,14.116,13.859,12.046,8.996,15.791,15.186,14.334,0.028,0.039,0.0,0.498,0.081,0.101,0.071,1519124201351034124,0000,0,00nn,AAUC,-19,123,355,134,0.188,0.048688


First, store the observed colors and then define the Monte Carlo arrays for W1, W2 and W3, with the same colors but now for Monte Carlo arrays

In [16]:
new_allwise['W1-W2_obs'] = new_allwise['W1mag'] - new_allwise['W2mag']
new_allwise['W2-W3_obs'] = new_allwise['W2mag'] - new_allwise['W3mag']

In [17]:
w1 = wm.array_montecarlo(new_allwise['W1mag'],new_allwise['e_W1mag'])
w2 = wm.array_montecarlo(new_allwise['W2mag'],new_allwise['e_W2mag'])
w3 = wm.array_montecarlo(new_allwise['W3mag'],new_allwise['e_W3mag'])

w1w2_obs = w1-w2
w2w3_obs = w2-w3

# K-corrections for the defined sample

Now that all initial colors are created, one can start by the first masking of the reject zone by deleting all sources with a W2-W3>4.4

In [21]:
first_reject_zone = new_allwise['W2-W3_obs']<=4.4

In [22]:
w1 = w1[first_reject_zone]
w2 = w2[first_reject_zone]
w3 = w3[first_reject_zone]

w1w2_obs = w1w2_obs[first_reject_zone]
w2w3_obs = w2w3_obs[first_reject_zone]

new_allwise = new_allwise[first_reject_zone]

It is necessary to create new arrays to store K-corrected W1abs and the K-correction factor of W1-W2 and W2-W3

In [ ]:
w1w2_kcors = np.zeros(np.shape(w1w2)[0])                                                
w2w3_kcors = np.zeros(np.shape(w2w3)[0])
w1abs = np.zeros(np.shape(w1))

In [ ]:
object_condition = (allwise['NED_TYPE']=='RadioS') | (allwise['NED_TYPE']=='QSO')
color_condition_1 = (allwise['W1-W2_obs']>0.8) & (allwise['W2-W3_obs']<2.2)
color_condition_2 = (allwise['W1-W2_obs']>wm.w1w2_treshold_qso(allwise['W2-W3_obs'])) & (allwise['W2-W3_obs']>=2.2) & (allwise['W2-W3_obs']<=4.4)

optimal_sample = allwise[(allwise['Z']<0.5) & ~object_condition & ~(color_condition_1 | color_condition_2)]
suboptimal_sample = allwise[(allwise['Z']>=0.5) & (allwise['Z']<=3) & ~object_condition & ~(color_condition_1 | color_condition_2)]  #Samples for k-correcion                
nok_sample = allwise[(allwise['Z']>3) | object_condition | color_condition_1 | color_condition_2]
                                                                                        
allwise.loc[optimal_sample.index, 'K_QUALITY'] = int(0)
allwise.loc[suboptimal_sample.index, 'K_QUALITY'] = int(1)
allwise.loc[nok_sample.index, 'K_QUALITY'] = int(2)